In [1]:
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.models as km
import tensorflow.keras.layers as kl
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display as display
tf.enable_eager_execution()

In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/sample_submission.csv')
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
train_image_names = train['id']
ytrain = train['has_cactus']

In [4]:
train_image_names.values

array(['0004be2cfeaba1c0361d39e2b000257b.jpg',
       '000c8a36845c0208e833c79c1bffedd1.jpg',
       '000d1e9a533f62e55c289303b072733d.jpg', ...,
       'fff059ecc91b30be5745e8b81111dc7b.jpg',
       'fff43acb3b7a23edcc4ae937be2b7522.jpg',
       'fffd9e9b990eba07c836745d8aef1a3a.jpg'], dtype=object)

In [5]:
train_image_paths = '../input/train/train/'+ train_image_names.values

In [6]:
train_image_paths

array(['../input/train/train/0004be2cfeaba1c0361d39e2b000257b.jpg',
       '../input/train/train/000c8a36845c0208e833c79c1bffedd1.jpg',
       '../input/train/train/000d1e9a533f62e55c289303b072733d.jpg', ...,
       '../input/train/train/fff059ecc91b30be5745e8b81111dc7b.jpg',
       '../input/train/train/fff43acb3b7a23edcc4ae937be2b7522.jpg',
       '../input/train/train/fffd9e9b990eba07c836745d8aef1a3a.jpg'],
      dtype=object)

In [7]:
image = train_image_paths[0]

In [8]:
ytrain.values

array([1, 1, 1, ..., 1, 0, 1])

In [46]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize_images(image,[32,32])
    image /= 255.0
    return image

In [47]:
def load_and_preprocess_image(path):
    image = tf.read_file(path)
    return preprocess_image(image)

In [48]:
path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)

In [49]:
ds = path_ds.map(load_and_preprocess_image,num_parallel_calls=AUTOTUNE)

In [50]:
labels_ds = tf.data.Dataset.from_tensor_slices(ytrain)

In [51]:
ds_label_ds = tf.data.Dataset.zip((ds,labels_ds))

In [52]:
ds_label_ds = ds_label_ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=len(train)))

In [53]:
ds_label_ds = ds_label_ds.batch(30)
ds_label_ds = ds_label_ds.prefetch(buffer_size=AUTOTUNE)

In [54]:
ds_label_ds

<DatasetV1Adapter shapes: ((?, 32, 32, 3), (?,)), types: (tf.float32, tf.int32)>

In [55]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [60]:
model = km.Sequential([
    kl.Conv2D(filters=32, kernel_size=3,padding='same',input_shape=(32,32,3),activation=tf.nn.relu),    
])

In [61]:
model.add(kl.Conv2D(32, (3, 3)))
model.add(kl.Activation('relu'))
model.add(kl.MaxPooling2D(pool_size=(2, 2)))
model.add(kl.Dropout(0.25))

model.add(kl.Conv2D(64, (3, 3), padding='same'))
model.add(kl.Activation('relu'))
model.add(kl.Conv2D(64, (3, 3)))
model.add(kl.Activation('relu'))
model.add(kl.MaxPooling2D(pool_size=(2, 2)))
model.add(kl.Dropout(0.25))

model.add(kl.Conv2D(64, (3, 3), padding='same'))
model.add(kl.Activation('relu'))
model.add(kl.Conv2D(64, (3, 3)))
model.add(kl.Activation('relu'))
model.add(kl.MaxPooling2D(pool_size=(2, 2)))
model.add(kl.Dropout(0.25))

model.add(kl.Flatten())
model.add(kl.Dense(512))
model.add(kl.Activation('relu'))
model.add(kl.Dropout(0.5))
model.add(kl.Dense(2))
model.add(kl.Activation('softmax'))

In [62]:
model.compile(optimizer='adam',loss=tf.keras.losses.sparse_categorical_crossentropy,
             metrics=['accuracy'])

In [63]:
model.fit(ds_label_ds,epochs=5,steps_per_epoch=len(train)//5)

Epoch 1/5
3500/3500 [==============================] - 54s 15ms/step - loss: 0.0767 - acc: 0.9714
Epoch 2/5
3500/3500 [==============================] - 47s 13ms/step - loss: 0.0335 - acc: 0.9884
Epoch 3/5
3500/3500 [==============================] - 47s 13ms/step - loss: 0.0226 - acc: 0.9923
Epoch 4/5
3500/3500 [==============================] - 47s 14ms/step - loss: 0.0183 - acc: 0.9941
Epoch 5/5
3500/3500 [==============================] - 47s 13ms/step - loss: 0.0141 - acc: 0.9954


In [64]:
test.shape

(4000, 2)

In [65]:
test_image_names = test['id']

In [66]:
test_image_paths = '../input/test/test/'+test_image_names

In [73]:
test_image_paths.values

array(['../input/test/test/000940378805c44108d287872b2f04ce.jpg',
       '../input/test/test/0017242f54ececa4512b4d7937d1e21e.jpg',
       '../input/test/test/001ee6d8564003107853118ab87df407.jpg', ...,
       '../input/test/test/ffbd469c56873d064326204aac546e0d.jpg',
       '../input/test/test/ffcb76b7d47f29ece11c751e5f763f52.jpg',
       '../input/test/test/fffed17d1a8e0433a934db518d7f532c.jpg'],
      dtype=object)

In [74]:
Xtest = []

In [75]:
import cv2

In [76]:
for path in test_image_paths:
    image = cv2.imread(path)
    Xtest.append(image)

In [77]:
Xtest = np.reshape(Xtest,newshape=(-1,32,32,3))
Xtest = Xtest / 255.0

In [78]:
test_ds = tf.data.Dataset.from_tensor_slices(Xtest)

In [79]:
test_ds = test_ds.batch(30)

In [80]:
pre = model.predict(test_ds,steps=len(test))

In [81]:
pre.shape

(4000, 2)

In [82]:
pre_ = np.argmax(pre,axis=1)

In [83]:
pre_

array([1, 1, 0, ..., 1, 1, 1])

In [84]:
test.has_cactus = pre_

In [86]:
test.to_csv('submission_7.csv',index=False)